# Spelling checker based on Confusion Matrix and Bayes rule

In [1]:
import re
import math
from collections import Counter
import pandas as pd

In [2]:
#Read from count_1w
#The first colum refer to the word as list of key
#The second colum refer to the number the word appears as list of occurrency
#Make COUNTS as a dictionary
TEXT = pd.read_csv("count_1w.csv")
BAG = open('big.txt').read()
key = TEXT['KEY']
key = list(key)
occurrency = TEXT['OCCURRENCY']
occurrency = list(occurrency)
COUNTS = dict(zip(key, occurrency))

Language model

In [3]:
#Get the probability of the occurence of a word P(w)
total = sum(occurrency)

def pdist(counter):
    N = total
    return lambda x: COUNTS[x]/N if x in key else 0/N
  
P = pdist(COUNTS)

In [4]:
def tokens(text):
    return re.findall('[a-z]+', text.lower()) 
#test the probability
for w in tokens('"The" test haha, you outrivaled unmentioned of actress'):
    print(P(w), w)


0.0393383750709 the
0.000263549062732 test
7.20235428946e-06 haha
0.00509446970922 you
0.0 outrivaled
1.01650294186e-07 unmentioned
0.0223625253383 of
1.19193458786e-05 actress


In [5]:
#use 'big.txt' as base document
WORDS = tokens(BAG)
WORDBAG = Counter(WORDS)

Error model

In [6]:
#4 matrices, reference confusion matrices in csv based on spell-errors.txt
#Statistic calculated the error occurence for each aphabete
#The first column refer to the the combination of 2 letters and the second column refer to the occurence that it happens
ERRORS = open('spell-errors.txt').read()
#Deletion matrix 26x26 
delmatrix = pd.read_csv('matrices/delconfusion.csv')
delmatrix_dict = dict(zip(delmatrix['XY'], delmatrix['COUNT']))
#Insertion matrix 26x26 
insmatrix = pd.read_csv('matrices/insconfusion.csv')
insmatrix_dict = dict(zip(insmatrix['XY'], insmatrix['COUNT']))
#Substitution matrix 26x26 
submatrix = pd.read_csv('matrices/subconfusion.csv')
submatrix_dict = dict(zip(submatrix['XY'], submatrix['COUNT']))
#Transition matrix
transmatrix = pd.read_csv('matrices/transconfusion.csv')
transmatrix_dict = dict(zip(transmatrix['XY'], transmatrix['COUNT']))

In [7]:
#Calculate p(x|w) based on 4 different situations. 
def maxpossib(x,y,opr):
    if opr == 'del':
        if x+y in delmatrix_dict:
            val = delmatrix_dict[x+y]/BAG.count(x+y) 
        else:
            val = 0
    if opr == 'ins':
        if x == '#':
            val = insmatrix_dict[x+y]/BAG.count(' '+y) 
        elif x+y in insmatrix_dict:
            val = insmatrix_dict[x+y]/BAG.count(x) 
        else:
            val = 0
    if opr == 'sub':
        if x+y in submatrix_dict:
            val = submatrix_dict[(x+y)[0:2]]/BAG.count(y) 
        else:
            val = 0
    if opr == 'trans':
        if x+y in transmatrix_dict:
            val = transmatrix_dict[x+y]/BAG.count(x+y)
        else:
            val = 0
    return val
maxpossib('a','y','sub')

5.6691913465463287e-05

In [8]:
#Determine the error type and return mis-spell related chars in order to get count numbers from confusion matrixes
def errortype(candidate, word):
    sizec = len(candidate)
    sizew = len(word)
    size = min(sizec,sizew)
    delete = False
    ins = False 
    sub = False 
    trans = False
    c1 = ''
    c2 = ''
    for i in range(0, size-1):
        if candidate[0:i+1] != word[0:i+1]:
            # a char missing, delete
            if candidate[i:] == word[i-1:]:
                delete = True
                c1 = candidate[i-2]
                c2 = candidate[i-1]
                break
            # a char adding, insertion
            elif candidate[i:] == word[i+1:]:
                ins = True
                if i == 0:
                    c1 = '#'
                    c2 = word[i]
                else:
                    c1 = candidate[i]
                    c2 = word[i]
                break
            # only one char different, substitution
            elif candidate[i+1:] == word[i+1:]:
                sub = True
                c1 = word[i]
                c2 = candidate[i]
                break
            # only order different, transition
            elif candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]:
                trans = True
                c1 = candidate[i+1]
                c2 = word[i+1]
                break
    #last char different
    if(word != candidate and not (delete or ins or sub or trans)):          
        if sizec == sizew:
            sub = True
            c1 = word[sizew-1]
            c2 = candidate[sizec-1]
        elif sizec > sizew:
            delete = True
            c1 = candidate[sizec-2]
            c2 = candidate[sizec-1]
        elif sizec < sizew:
            ins = True
            c1 = '#'
            c2 = word[sizew-1]
              
            
    if word == candidate:
        return "correct", '', ''
    if delete:
        return "del", c1, c2
    elif ins:
        return "ins", c1, c2
    elif sub:
        return "sub", c1, c2
    elif trans:
        return "trans", c1, c2

print(errortype('guns', 'guic'))



('sub', 'c', 's')


In [9]:
#Reference functions from Lab5: define edit distance 0 1 or 2
#Define correct function, for each candidate word, calculate the val = p(x|w)*p(w) then return the max val and corresponding word
def correct(word):
    val = 0
    maxval = 0
    correct = ''
    #Generating all the words with edit distance of 0, 1 & 2
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    print(candidates)
    for candi in candidates:
        if len(candidates) == 1:
            correct = candi
        errors = errortype(candi, word)
        print(candi, errors)
        if errors == None: 
            continue
        if errors[0] == 'correct':
            correct = candi
        else:
            val = maxpossib(errors[1], errors[2], errors[0])*P(candi)
            print(val)
            if(val > maxval):
                maxval = val
                correct = candi
            
    print('maxval:', maxval)
         
    return correct
    

def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in WORDBAG}

def edits0(word): 
    return {word}

alphabet = 'abcdefghijklmnopqrstuvwxyz'
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

correct('acress')


{'access', 'acres', 'actress', 'caress', 'across'}
access ('sub', 'r', 'c')
0.0
acres ('ins', '#', 's')
8.86877521114e-09
actress ('del', 'c', 't')
4.90844717033e-08
caress ('trans', 'a', 'c')
1.26651435084e-10
across ('sub', 'e', 'o')
3.16509922985e-08
maxval: 4.90844717033e-08


'actress'

Test

In [10]:
doc_test = pd.read_csv("test.csv")
wrong_words = doc_test['WRONG']
wrong_list = list(wrong_words)

In [11]:
correct_list = list(map(correct, wrong_list))
pd.DataFrame(correct_list, columns=['CORRECT']).to_csv('prediction_matrix.csv')

{'contented'}
contented ('ins', 't', 'p')
1.70725687184e-12
maxval: 1.70725687184e-12
{'beginning'}
beginning ('del', 'i', 'n')
7.18833820261e-08
maxval: 7.18833820261e-08
{'problem'}
problem ('sub', 'a', 'e')
1.30524413572e-07
maxval: 1.30524413572e-07
{'driven'}
driven ('trans', 'i', 'r')
3.28802247954e-08
maxval: 3.28802247954e-08
{'eustace', 'ecstasy'}
eustace ('sub', 'y', 'e')
9.52704750513e-12
ecstasy ('sub', 'c', 's')
3.59328910244e-10
maxval: 3.59328910244e-10
{'eric', 'quick', 'cubic', 'qui', 'sic', 'guise', 'grice', 'gums', 'guai', 'guild', 'gut', 'quid', 'grin', 'gust', 'lui', 'juice', 'muir', 'grit', 'gulp', 'gig', 'gulf', 'grid', 'gum', 'quit', 'guam', 'guy', 'suis', 'suit', 'guide', 'music', 'guilt', 'grip', 'duc', 'gush', 'gulch', 'ruin', 'gin', 'glio', 'uric', 'oui', 'guile', 'gain', 'guns', 'gun', 'tic', 'epic', 'git', 'gait', 'tunic', 'grim'}
eric ('sub', 'u', 'r')
5.98900985656e-10
quick ('del', 'c', 'k')
2.5955047234e-07
cubic ('del', 'i', 'c')
2.63514138465e-08
qui

lash ('sub', 'g', 's')
5.48780267854e-11
laugh ('del', 'g', 'h')
4.11444480307e-08
maxval: 4.11444480307e-08
{'often', 'open', 'oxen', 'owen', 'oven', 'omen'}
often ('del', 'e', 'n')
2.1611295368e-07
open ('sub', 'f', 'p')
0.0
oxen ('sub', 'f', 'x')
0.0
owen ('sub', 'f', 'w')
2.18019360854e-10
oven ('sub', 'f', 'v')
0.0
omen ('sub', 'f', 'm')
4.55180968437e-11
maxval: 2.1611295368e-07
{'someone'}
someone ('del', 'm', 'e')
1.3627062831e-07
maxval: 1.3627062831e-07
{'personnel'}
personnel ('ins', '#', 'l')
1.29706336514e-08
maxval: 1.29706336514e-08
{'unique'}
unique ('sub', 'e', 'i')
2.74099774051e-08
maxval: 2.74099774051e-08
['diagrammaticaally']
diagrammaticaally ('correct', '', '')
maxval: 0
{'description'}
description ('sub', 'i', 'e')
7.75685026687e-08
maxval: 7.75685026687e-08
{'poems'}
poems ('sub', 'i', 'e')
5.00361521762e-09
maxval: 5.00361521762e-09
{'purple', 'people', 'perplex'}
purple ('sub', 'e', 'u')
2.95830083956e-09
people ('sub', 'r', 'o')
2.13406382687e-09
perplex ('

maxval: 3.77772052271e-07
{'central'}
central ('ins', '#', 'y')
2.38384066729e-08
maxval: 2.38384066729e-08
{'families'}
families ('del', 'e', 's')
3.73333539895e-07
maxval: 3.73333539895e-07
{'bicycle'}
bicycle ('trans', 'e', 'l')
1.54291917727e-08
maxval: 1.54291917727e-08
{'choice', 'chose', 'chaise', 'choose'}
choice ('sub', 's', 'c')
2.25562038839e-08
chose ('ins', 's', 'i')
4.95336387601e-09
chaise ('sub', 'o', 'a')
2.34390499145e-10
choose ('sub', 'i', 'o')
1.95965421713e-08
maxval: 2.25562038839e-08
{'opposite'}
opposite ('del', 'p', 'o')
5.36045240714e-08
maxval: 5.36045240714e-08
{'certains', 'captains', 'curtains'}
certains ('sub', 'a', 'e')
1.47603119018e-10
captains ('sub', 'r', 'p')
4.23294652078e-10
curtains ('sub', 'a', 'u')
3.80905834543e-10
maxval: 4.23294652078e-10
{'dress', 'address'}
dress ('ins', '#', 'a')
1.91732714757e-08
address ('del', 'a', 'd')
4.92838610379e-07
maxval: 4.92838610379e-07
{'liaison'}
liaison ('ins', 'o', 'i')
6.26068734602e-10
maxval: 6.260687

7.67076108443e-09
maxval: 7.67076108443e-09
{'possibilities'}
possibilities ('del', 'b', 'i')
1.06193288471e-06
maxval: 1.06193288471e-06
{'building'}
building ('trans', 'i', 'u')
0.0
maxval: 0
{'universally', 'universal'}
universally ('del', 'l', 'y')
1.49048480737e-09
universal ('ins', '#', 'y')
7.85110143047e-09
maxval: 7.85110143047e-09
['unessasarily']
unessasarily ('correct', '', '')
maxval: 0
{'beginning'}
beginning ('del', 'i', 'n')
7.18833820261e-08
maxval: 7.18833820261e-08
{'ant', 'an', 'and', 'any', 'ann', 'ana', 'ano'}
ant ('sub', 'f', 't')
2.83830414381e-10
an ('ins', '#', 'f')
7.9434247291e-07
and ('sub', 'f', 'd')
3.13215289609e-07
any ('sub', 'f', 'y')
0.0
ann ('sub', 'f', 'n')
1.3527638348e-10
ana ('sub', 'f', 'a')
0.0
ano ('sub', 'f', 'o')
0.0
maxval: 7.9434247291e-07
{'moments', 'comments'}
moments ('sub', 'c', 'm')
1.20388106904e-09
comments ('del', 'o', 'm')
1.93682886317e-07
maxval: 1.93682886317e-07
{'thaw', 'the', 'th', 'thy', 'tow'}
thaw ('del', 'a', 'w')
0.0


chaises ('sub', 'o', 'a')
3.30297348616e-11
maxval: 7.04962742465e-10
{'conference'}
conference ('sub', 'a', 'e')
9.44024713758e-08
maxval: 9.44024713758e-08
{'preceding', 'proceeding'}
preceding ('sub', 'o', 'e')
2.02543711889e-09
proceeding ('del', 'c', 'e')
3.05334835434e-08
maxval: 3.05334835434e-08
{'profits'}
profits ('ins', 'i', 'f')
0.0
maxval: 0
{'apparent'}
apparent ('sub', 'a', 'e')
8.93266422289e-09
maxval: 8.93266422289e-09
{'ideals', 'ideally', 'ideal'}
ideals ('sub', 'y', 's')
4.56721442033e-10
ideally ('del', 'l', 'y')
4.00043018187e-09
ideal ('ins', '#', 'y')
5.45197667765e-09
maxval: 5.45197667765e-09
{'ability'}
ability ('del', 'b', 'i')
6.6700699949e-06
maxval: 6.6700699949e-06
{'responsibilities'}
responsibilities ('del', 'b', 'i')
2.14287723586e-06
maxval: 2.14287723586e-06
{'alte', 'ante', 'rate', 'ate', 'carte', 'acte', 'are', 'arts', 'art'}
alte ('sub', 'r', 'l')
4.23104149708e-11
ante ('sub', 'r', 'n')
1.87107055825e-10
rate ('trans', 'a', 'r')
2.39154216015e-